In [11]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil import parser




In [12]:
solar = pd.read_csv('Eclipse_Data/solar.csv')
lunar = pd.read_csv('Eclipse_Data/lunar.csv')

In [13]:
lunar.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Quincena Solar Eclipse,Gamma,Penumbral Magnitude,Umbral Magnitude,Latitude,Longitude,Penumbral Eclipse Duration (m),Partial Eclipse Duration (m),Total Eclipse Duration (m)
0,1,-1999 June 26,14:13:28,46437,-49456,17,N,t-,-1.0981,0.8791,-0.1922,24S,22W,268.8,-,-
1,2,-1999 November 21,20:23:49,46427,-49451,-16,N,-a,-1.1155,0.8143,-0.1921,15N,98W,233.4,-,-
2,3,-1998 May 17,05:47:36,46416,-49445,-11,P,-t,0.8988,1.2105,0.2069,13S,89E,281.7,102.7,-
3,4,-1998 November 11,05:15:58,46404,-49439,-6,P,-a,-0.4644,2.0382,0.9740,12N,113E,343.4,200.8,-
4,5,-1997 May 6,18:57:01,46392,-49433,-1,T+,pp,0.1003,2.6513,1.6963,11S,92W,322.8,213.5,98.2


In [14]:
lunar_sub_df = lunar.drop(lunar.columns[7:16], axis=1)
lunar_sub_df = lunar_sub_df.drop(lunar_sub_df.columns[4:6], axis=1)
lunar_sub_df.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Eclipse Type
0,1,-1999 June 26,14:13:28,46437,N
1,2,-1999 November 21,20:23:49,46427,N
2,3,-1998 May 17,05:47:36,46416,P
3,4,-1998 November 11,05:15:58,46404,P
4,5,-1997 May 6,18:57:01,46392,T+


In [15]:
solar.head()
solar_sub_df = solar.drop(solar.columns[7:16], axis=1)
solar_sub_df = solar_sub_df.drop(solar_sub_df.columns[4:6], axis=1)
solar_sub_df.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Eclipse Type
0,1,-1999 June 12,03:14:51,46438,T
1,2,-1999 December 5,23:45:23,46426,A
2,3,-1998 June 1,18:09:16,46415,T
3,4,-1998 November 25,05:57:03,46403,A
4,5,-1997 April 22,13:19:56,46393,P


In [16]:
solar_sub_df['Overall Type'] = 'solar'
solar_sub_df.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Eclipse Type,Overall Type
0,1,-1999 June 12,03:14:51,46438,T,solar
1,2,-1999 December 5,23:45:23,46426,A,solar
2,3,-1998 June 1,18:09:16,46415,T,solar
3,4,-1998 November 25,05:57:03,46403,A,solar
4,5,-1997 April 22,13:19:56,46393,P,solar


In [17]:
lunar_sub_df['Overall Type'] = 'lunar'
lunar_sub_df.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Eclipse Type,Overall Type
0,1,-1999 June 26,14:13:28,46437,N,lunar
1,2,-1999 November 21,20:23:49,46427,N,lunar
2,3,-1998 May 17,05:47:36,46416,P,lunar
3,4,-1998 November 11,05:15:58,46404,P,lunar
4,5,-1997 May 6,18:57:01,46392,T+,lunar


In [18]:
merged_df = pd.concat([solar_sub_df, lunar_sub_df])
merged_df.dtypes

Catalog Number     int64
Calendar Date     object
Eclipse Time      object
Delta T (s)        int64
Eclipse Type      object
Overall Type      object
dtype: object

In [23]:
positive_yr_df = merged_df[~merged_df['Calendar Date'].str.startswith('-')]


(14371, 6)

In [57]:
def convert_to_datetime(date_str, time_str):
    try:
        # Check if the year is a single digit
        if len(date_str.split()[0]) == 1:
            year = f"000{date_str.split()[0]}"
        if len(date_str.split()[0]) == 2:
            year = f"00{date_str.split()[0]}"
        if len(date_str.split()[0]) == 3:
            year = f"0{date_str.split()[0]}"
        if len(date_str.split()[0]) == 4:
            year = f"{date_str.split()[0]}"
        
        date_time_str = f"{year} {date_str.split()[2]} {date_str.split()[1]} {time_str}"
        return datetime.strptime(date_time_str, '%Y %d %B %H:%M:%S')

    except ValueError:
        return pd.NaT  # Return NaT (Not a Time) for invalid dates

# Convert "Calendar Date" and "Eclipse Time" to datetime objects
positive_yr_df['Combined Datetime'] = positive_yr_df.apply(lambda row: convert_to_datetime(row['Calendar Date'], row['Eclipse Time']), axis=1)
positive_yr_df = positive_yr_df.dropna(subset=['Combined Datetime'])

0000 20 June 13:45:32
0000 19 July 20:44:11
0000 14 December 18:19:36
0001 10 June 06:44:16
0001 3 December 20:05:38
0002 30 May 19:45:43
0002 23 November 05:06:15
0003 20 May 01:41:30
0003 14 October 09:21:59
0003 12 November 19:36:29
0004 8 April 11:39:14
0004 2 October 23:52:25
0005 28 March 17:23:34
0005 22 September 08:42:23
0006 18 March 06:05:10
0006 11 September 10:36:21
0007 6 February 13:24:00
0007 7 March 22:28:54
0007 1 August 20:02:56
0007 31 August 10:35:56
0008 27 January 02:17:23
0008 21 July 05:28:07
0009 15 January 08:08:55
0009 10 July 20:53:09
0010 4 January 07:54:31
0010 30 June 14:01:10
0010 24 November 16:30:45
0010 24 December 08:29:33
0011 21 May 17:47:07
0011 14 November 03:29:00
0012 9 May 21:45:58
0012 2 November 18:48:38
0013 28 April 22:16:18
0013 23 October 09:50:43
0014 19 March 15:38:24
0014 18 April 02:41:51
0014 13 September 04:52:58
0014 12 October 20:04:10
0015 9 March 05:44:12
0015 2 September 05:34:31
0016 26 February 22:16:04
0016 21 August 06:33

/var/folders/gk/wklnyjmd5mscj9kxvb5nbxbw0000gn/T/ipykernel_49792/2070988165.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_yr_df['Combined Datetime'] = positive_yr_df.apply(lambda row: convert_to_datetime(row['Calendar Date'], row['Eclipse Time']), axis=1)


In [58]:
positive_yr_df.head(20)

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Eclipse Type,Overall Type,Combined Datetime
4774,4775,1 June 10,06:44:16,10520,T,solar,0001-06-10 06:44:16
4775,4776,1 December 3,20:05:38,10515,A,solar,0001-12-03 20:05:38
4776,4777,2 May 30,19:45:43,10510,H,solar,0002-05-30 19:45:43
4777,4778,2 November 23,05:06:15,10505,H,solar,0002-11-23 05:06:15
4778,4779,3 May 20,01:41:30,10500,P,solar,0003-05-20 01:41:30
4779,4780,3 October 14,09:21:59,10496,P,solar,0003-10-14 09:21:59
4780,4781,3 November 12,19:36:29,10495,P,solar,0003-11-12 19:36:29
4781,4782,4 April 8,11:39:14,10491,A,solar,0004-04-08 11:39:14
4782,4783,4 October 2,23:52:25,10486,H,solar,0004-10-02 23:52:25
4783,4784,5 March 28,17:23:34,10481,H,solar,0005-03-28 17:23:34


In [67]:
positive_yr_sorted = positive_yr_df.sort_values(by='Combined Datetime').reset_index(drop=True)
positive_yr_sorted.head(20)

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Eclipse Type,Overall Type,Combined Datetime
0,4775,1 June 10,06:44:16,10520,T,solar,0001-06-10 06:44:16
1,4824,1 June 24,12:08:47,10519,P,lunar,0001-06-24 12:08:47
2,4825,1 November 19,19:31:30,10515,N,lunar,0001-11-19 19:31:30
3,4776,1 December 3,20:05:38,10515,A,solar,0001-12-03 20:05:38
4,4826,1 December 19,08:21:56,10514,N,lunar,0001-12-19 08:21:56
5,4827,2 May 15,07:24:42,10510,P,lunar,0002-05-15 07:24:42
6,4777,2 May 30,19:45:43,10510,H,solar,0002-05-30 19:45:43
7,4828,2 November 9,01:38:13,10505,P,lunar,0002-11-09 01:38:13
8,4778,2 November 23,05:06:15,10505,H,solar,0002-11-23 05:06:15
9,4829,3 May 4,22:22:05,10501,T-,lunar,0003-05-04 22:22:05


In [68]:
positive_yr_sorted['Time Difference'] = None

# Iterate through the rows to calculate time differences
for i in range(1, len(positive_yr_sorted)):
    if (positive_yr_sorted.loc[i, 'Combined Datetime'] - positive_yr_sorted.loc[i - 1, 'Combined Datetime']).days <= 21:
        time_diff = positive_yr_sorted.loc[i, 'Combined Datetime'] - positive_yr_sorted.loc[i - 1, 'Combined Datetime']
        positive_yr_sorted.at[i, 'Time Difference'] = time_diff

In [75]:
positive_yr_sorted.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Eclipse Type,Overall Type,Combined Datetime,Time Difference
0,4775,1 June 10,06:44:16,10520,T,solar,0001-06-10 06:44:16,None
1,4824,1 June 24,12:08:47,10519,P,lunar,0001-06-24 12:08:47,"14 days, 5:24:31"
2,4825,1 November 19,19:31:30,10515,N,lunar,0001-11-19 19:31:30,None
3,4776,1 December 3,20:05:38,10515,A,solar,0001-12-03 20:05:38,"14 days, 0:34:08"
4,4826,1 December 19,08:21:56,10514,N,lunar,0001-12-19 08:21:56,"15 days, 12:16:18"


In [70]:
positive_yr_sorted.to_csv('eclipse_diff.csv')

In [73]:

positive_yr_sorted['Year'] = positive_yr_sorted['Combined Datetime'].dt.year

before_2024 = positive_yr_sorted[positive_yr_sorted['Year'] <= 2024]
after_2024 = positive_yr_sorted[positive_yr_sorted['Year'] > 2024]

AttributeError: Can only use .dt accessor with datetimelike values

In [78]:
avg_difference= positive_yr_sorted['Time Difference'].groupby(positive_yr_sorted['Eclipse Type']).mean()
print(avg_difference)

Eclipse Type
A     14 days 16:42:21.647605432
A+    14 days 23:05:22.454545454
A-    14 days 14:32:40.363636363
Am    14 days 16:53:13.355555555
An    14 days 18:28:04.642857142
As    14 days 13:28:49.333333333
H     14 days 18:04:24.541176470
H2    14 days 00:57:53.941176470
H3    15 days 12:50:23.666666666
Hm    14 days 04:20:29.428571428
N     14 days 18:18:02.779060181
Nb    14 days 17:24:16.530303030
Ne                           NaT
Nx    14 days 17:47:37.694444444
P     14 days 18:16:10.579489291
Pb    14 days 17:15:26.907216494
Pe                           NaT
T     14 days 19:09:43.497931034
T+    14 days 16:54:07.031067961
T-    14 days 16:29:06.143145161
Tm    14 days 20:30:14.545454545
Tn    14 days 15:57:23.333333333
Ts    14 days 16:44:21.666666666
Name: Time Difference, dtype: object
